<a href="https://colab.research.google.com/github/Snowdenstyll/Lottery/blob/main/server%5CjupyterNotebooks%5CKeno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("2024.csv")

In [3]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582 entries, 0 to 581
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  582 non-null    object
 1   AP        582 non-null    object
 2   N01       582 non-null    int64 
 3   N02       582 non-null    int64 
 4   N03       582 non-null    int64 
 5   N04       582 non-null    int64 
 6   N05       582 non-null    int64 
 7   N06       582 non-null    int64 
 8   N07       582 non-null    int64 
 9   N08       582 non-null    int64 
 10  N09       582 non-null    int64 
 11  N10       582 non-null    int64 
 12  N11       582 non-null    int64 
 13  N12       582 non-null    int64 
 14  N13       582 non-null    int64 
 15  N14       582 non-null    int64 
 16  N15       582 non-null    int64 
 17  N16       582 non-null    int64 
 18  N17       582 non-null    int64 
 19  N18       582 non-null    int64 
 20  N19       582 non-null    int64 
 21  N20       582 no

In [4]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [5]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 5
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [6]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [7]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 32
ltsm_units = 256
dropout = 0.2

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(dropout))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(dropout))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(ltsm_units,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(dropout))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.89, random_state=89)

# Train the model
#history = model.fit(X_train, y_train, batch_size=batch_size, epochs=200, verbose=2)
history = model.fit(X_train, y_train, batch_size=30, epochs=600, verbose=2)

Epoch 1/600
3/3 - 11s - 4s/step - accuracy: 0.0317 - loss: 0.9394
Epoch 2/600
3/3 - 0s - 25ms/step - accuracy: 0.1270 - loss: 0.8953
Epoch 3/600
3/3 - 0s - 46ms/step - accuracy: 0.1429 - loss: 0.8484
Epoch 4/600
3/3 - 0s - 45ms/step - accuracy: 0.1429 - loss: 0.8265
Epoch 5/600
3/3 - 0s - 46ms/step - accuracy: 0.1429 - loss: 0.7841
Epoch 6/600
3/3 - 0s - 24ms/step - accuracy: 0.1746 - loss: 0.7979
Epoch 7/600
3/3 - 0s - 27ms/step - accuracy: 0.1429 - loss: 0.7612
Epoch 8/600
3/3 - 0s - 43ms/step - accuracy: 0.0952 - loss: 0.7560
Epoch 9/600
3/3 - 0s - 46ms/step - accuracy: 0.1429 - loss: 0.7404
Epoch 10/600
3/3 - 0s - 22ms/step - accuracy: 0.1270 - loss: 0.7311
Epoch 11/600
3/3 - 0s - 21ms/step - accuracy: 0.1429 - loss: 0.6733
Epoch 12/600
3/3 - 0s - 46ms/step - accuracy: 0.1587 - loss: 0.7320
Epoch 13/600
3/3 - 0s - 45ms/step - accuracy: 0.1746 - loss: 0.6370
Epoch 14/600
3/3 - 0s - 22ms/step - accuracy: 0.1905 - loss: 0.6263
Epoch 15/600
3/3 - 0s - 22ms/step - accuracy: 0.1746 - los

In [17]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1457 - loss: 1.2827
Test Loss: 1.2533155679702759, Test Accuracy: 0.16464471817016602
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0658 - loss: 1.3345
Test Loss: 1.4058122634887695, Test Accuracy: 0.07392995804548264


In [10]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [11]:
import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'{label}.keras')
print("saved model")

saved model


In [12]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-12-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
577,2,4,6,8,15,17,18,19,21,23,31,32,34,45,46,47,49,54,67,68
578,9,16,19,21,23,26,27,31,32,36,40,41,46,48,49,51,52,53,54,58
579,4,6,8,11,17,22,25,28,31,33,46,48,49,53,55,63,64,66,68,70
580,15,21,22,28,30,33,37,38,39,43,46,49,53,55,60,61,63,66,67,68


In [13]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 521ms/step
The top 5 indices: [13 18 12 19 14]
The top 5 numbers: [45, 64, 44, 67, 46]


The top 10 indices: [13 18 12 19 14 17 16 15  6 11]
The top 10 numbers: [45, 64, 44, 67, 46, 56, 53, 47, 28, 41]


The predicted numbers for the lottery game are:   [ 1  5  7 12 16 19 23 24 27 31 34 39 45 48 50 53 57 60 64 67]
The actual numbers in the last lottery game were: [ 4 10 11 13 24 26 28 31 32 35 39 41 44 45 46 47 53 56 64 67]


The winning indices: [ 4  7 10 13 16 18 19]
winners: [24 31 39 45 53 64 67]
0.35


# Predict the next set of numbers

In [14]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
The predicted numbers for the lottery game: [ 1  5  7 12 16 19 23 24 27 31 34 39 45 48 50 53 57 60 64 67]


The top 5 indices [13 12 14 15 18]
The top 5 numbers: [52, 47, 53, 56, 65]


The top 10 indices [13 12 14 15 18 16 19 17  6  5]
The top 10 numbers: [52, 47, 53, 56, 65, 58, 68, 61, 24, 20]


winning indices [ 4  7 10 13 16 18 19]
winning numbers [16, 25, 32, 52, 58, 65, 68]


In [15]:
next = df.copy()
next = next.tail(7)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The top 5 indices [ 6  9 10  7  8]
The top 5 numbers: [25, 35, 38, 27, 30]


The top 10 indices [ 6  9 10  7  8 13 11 14  0  2]
The top 10 numbers: [25, 35, 38, 27, 30, 46, 39, 48, 2, 8]


winning indices [ 4  7 10 13 16 18 19]
winning numbers [14, 27, 38, 46, 54, 60, 65]


The predicted numbers: [[ 2  4  8  9 14 17 25 27 30 35 38 39 41 46 48 50 54 58 60 65]]
